# BUILD CHROMA CQT

In [1]:
import numpy as np
import librosa
import pandas as pd
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

### GLOBAL VARIABLES

In [2]:
INPUT_PATH = 'data/processed/submission.npy'
OUTPUT_PATH = 'data/features/submission_chromacqt.npy'
SR = 22050

### FUNCTIONS

In [3]:
def wave2mel(wave, sr, hop_length, max_pad_len, normalize, padding_mode):

    #wave, sr = librosa.load(wav_file, mono=True)
    
    if 0 < len(wave): # workaround: 0 length causes error
        wave, _ = librosa.effects.trim(wave) # trim, top_db=default(60)

    if normalize:
        wave = librosa.util.normalize(wave) # normalizing data before mfcc

    # making melspect from signal
    S = librosa.feature.chroma_cqt(y=wave, sr=sr, hop_length=hop_length)

    if max_pad_len:
        if S.shape[1] > max_pad_len:
            S = S[:,:max_pad_len]
        else:
            pad_width = max_pad_len - S.shape[1]
            S = np.pad(S, pad_width=((0, 0), (0, pad_width)), mode=padding_mode)

   
    return S

In [4]:
def gen_chroma(input_path, output_path, sr, hop_length=512, max_pad_len=None, normalize=True, padding_mode='constant'):
    
    # Load wave files
    wavfiles = np.load(input_path)
    
    # Init Mel vectors
    X_chroma = []
    
    for idx in trange(wavfiles.shape[0]):
        X_chroma.append(wave2mel(wavfiles[idx, :], sr, hop_length=hop_length, max_pad_len=max_pad_len, 
                             normalize=normalize, padding_mode=padding_mode))
        pass
    
    X_chroma = np.array(X_chroma)
    
    # Saving Mels 
    np.save(output_path, X_chroma)
    
    pass

### RUNING

In [5]:
gen_chroma(INPUT_PATH, OUTPUT_PATH, SR, hop_length=512, normalize=False)